## Predicting Kobe's shots using XGBOOST##

This notebook built on the analysis in: https://www.kaggle.com/selfishgene/kobe-bryant-shot-selection/psychology-of-a-professional-athlete

In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb

### Reading data

In [15]:
data = pd.read_csv('../input/data.csv')

Let's prepare the data for the splitting into train/test set later.

In [16]:
unknown_mask = data['shot_made_flag'].isnull()
data_cl = data.copy()

The following columns are to be dropped:

In [17]:
data_cl.drop('team_id', inplace=True, axis=1) #only 1 category
data_cl.drop('lat', inplace=True, axis=1) # correlated with loc_x
data_cl.drop('lon', inplace=True, axis=1) # correlated with loc_y
data_cl.drop('game_id', inplace=True, axis=1) # should not be dependent on game id, furthermore it's contained in opponent/match
data_cl.drop('game_event_id', inplace=True, axis=1) # independent, unique for every shots in a game
data_cl.drop('team_name', inplace=True, axis=1) # always LA Lakers

### Feature Engineering ##

As Kobe's psychology is impacted by the number of seconds from the game, let's create more features for these.

In [18]:
# time into the game
data_cl['seconds_from_period_end'] = 60 * data_cl['minutes_remaining'] + data_cl['seconds_remaining']
data_cl['last_5_sec_in_period'] = data_cl['seconds_from_period_end'] < 5
data_cl['seconds_from_period_start'] = 60*(11-data_cl['minutes_remaining'])+(60-data_cl['seconds_remaining'])
data_cl['seconds_from_game_start'] = (data_cl['period'] <= 4).astype(int)*(data_cl['period']-1)*12*60 + (data_cl['period'] > 4).astype(int)*((data_cl['period']-4)*5*60 + 3*12*60) + data_cl['seconds_from_period_start']

# drop redundant features
data_cl.drop('minutes_remaining', axis=1, inplace=True)
data_cl.drop('seconds_remaining', axis=1, inplace=True)

Matchup -- away/home:

In [19]:
data_cl['home_play'] = data_cl['matchup'].str.contains('vs').astype('int')
data_cl.drop('matchup', axis=1, inplace=True)

Extract year and month from date of game:

In [20]:
data_cl['game_date'] = pd.to_datetime(data_cl['game_date'])

# year and month
data_cl['game_year'] = data_cl['game_date'].dt.year
data_cl['game_month'] = data_cl['game_date'].dt.month

# day of week/year
data_cl['dayOfWeek'] = data_cl['game_date'].dt.dayofweek
data_cl['dayOfYear'] = data_cl['game_date'].dt.dayofyear

# make cyclical
data_cl['dayOfWeek_cycX'] = np.sin(2*np.pi*(data_cl['dayOfWeek']/7))
data_cl['dayOfWeek_cycY'] = np.cos(2*np.pi*(data_cl['dayOfWeek']/7))
data_cl['timeOfYear_cycX'] = np.sin(2*np.pi*(data_cl['dayOfYear']/365))
data_cl['timeOfYear_cycY'] = np.cos(2*np.pi*(data_cl['dayOfYear']/365))

data_cl.drop('dayOfWeek', axis=1, inplace=True)
data_cl.drop('dayOfYear', axis=1, inplace=True)

Create a flag whether or not the last shot by Kobe was hit or miss:

In [21]:
#%% collect data given that kobe made or missed last shot

timeBetweenShotsDict = {}
timeBetweenShotsDict['madeLast'] = []
timeBetweenShotsDict['missedLast'] = []

changeInDistFromBasketDict = {}
changeInDistFromBasketDict['madeLast'] = []
changeInDistFromBasketDict['missedLast'] = []

changeInShotDifficultyDict = {}
changeInShotDifficultyDict['madeLast'] = []
changeInShotDifficultyDict['missedLast'] = []

afterMadeShotsList = []
afterMissedShotsList = []

for shot in range(1,data_cl.shape[0]):

    # make sure the current shot and last shot were all in the same period of the same game
    sameGame   = data_cl.ix[shot,'game_date'] == data_cl.ix[shot-1,'game_date']
    samePeriod = data_cl.ix[shot,'period']    == data_cl.ix[shot-1,'period']

    if samePeriod and sameGame:
        madeLastShot       = data_cl.ix[shot-1,'shot_made_flag'] == 1
        missedLastShot     = data_cl.ix[shot-1,'shot_made_flag'] == 0
        
        timeDifferenceFromLastShot = data_cl.ix[shot,'seconds_from_game_start']     - data_cl.ix[shot-1,'seconds_from_game_start']
        distDifferenceFromLastShot = data_cl.ix[shot,'shot_distance']            - data_cl.ix[shot-1,'shot_distance']
        #shotDifficultyDifferenceFromLastShot = data_cl.ix[shot,'shotDifficulty'] - data_cl.ix[shot-1,'shot_difficulty']

        # check for currupt data points (assuming all samples should have been chronologically ordered)
        if timeDifferenceFromLastShot < 0:
            continue
        
        if madeLastShot:
            timeBetweenShotsDict['madeLast'].append(timeDifferenceFromLastShot)
            changeInDistFromBasketDict['madeLast'].append(distDifferenceFromLastShot)
            #changeInShotDifficultyDict['madeLast'].append(shotDifficultyDifferenceFromLastShot)
            afterMadeShotsList.append(shot)
            
        if missedLastShot:
            timeBetweenShotsDict['missedLast'].append(timeDifferenceFromLastShot)
            changeInDistFromBasketDict['missedLast'].append(distDifferenceFromLastShot)
            #changeInShotDifficultyDict['missedLast'].append(shotDifficultyDifferenceFromLastShot)
            afterMissedShotsList.append(shot)

data_cl['last_shot_miss'] = 0
data_cl.ix[afterMissedShotsList,'last_shot_miss'] = 1
data_cl['last_shot_made'] = 0
data_cl.ix[afterMadeShotsList,'last_shot_made'] = 1

data_cl.drop('game_date', axis=1, inplace=True)

Replace 20 least common action types with value 'Other'

In [22]:
rare_action_types = data_cl['action_type'].value_counts().sort_values().index.values[:20]
data_cl.loc[data_cl['action_type'].isin(rare_action_types), 'action_type'] = 'Other'

One-hot encoding of categorical variables:

In [23]:
categorial_cols = [
    'action_type', 'combined_shot_type', 'period', 'season', 'shot_type',
    'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'game_year',
    'game_month', 'opponent']

for cc in categorial_cols:
    dummies = pd.get_dummies(data_cl[cc])
    dummies = dummies.add_prefix("{}_".format(cc))
    data_cl.drop(cc, axis=1, inplace=True)
    data_cl = data_cl.join(dummies)

In [29]:
print(data_cl.columns.values)

['loc_x' 'loc_y' 'playoffs' 'shot_distance' 'shot_made_flag' 'shot_id'
 'seconds_from_period_end' 'last_5_sec_in_period'
 'seconds_from_period_start' 'seconds_from_game_start' 'home_play'
 'dayOfWeek_cycX' 'dayOfWeek_cycY' 'timeOfYear_cycX' 'timeOfYear_cycY'
 'last_shot_miss' 'last_shot_made' 'action_type_Alley Oop Dunk Shot'
 'action_type_Alley Oop Layup shot' 'action_type_Driving Dunk Shot'
 'action_type_Driving Finger Roll Layup Shot'
 'action_type_Driving Finger Roll Shot' 'action_type_Driving Jump shot'
 'action_type_Driving Layup Shot' 'action_type_Driving Reverse Layup Shot'
 'action_type_Driving Slam Dunk Shot' 'action_type_Dunk Shot'
 'action_type_Fadeaway Bank shot' 'action_type_Fadeaway Jump Shot'
 'action_type_Finger Roll Layup Shot' 'action_type_Finger Roll Shot'
 'action_type_Floating Jump shot' 'action_type_Follow Up Dunk Shot'
 'action_type_Hook Shot' 'action_type_Jump Bank Shot'
 'action_type_Jump Hook Shot' 'action_type_Jump Shot'
 'action_type_Layup Shot' 'action_typ

### Train/test/validation Splitting ##

In [30]:
data.set_index('shot_id', inplace=True)
# drop target variable
target = data_cl['shot_made_flag'].copy()
data_cl.drop('shot_made_flag', inplace=True, axis=1)

data_submit = data_cl[unknown_mask]
# Separate dataset for training
X = data_cl[~unknown_mask]
Y = target[~unknown_mask]

### XGBOOST

In [31]:
d_train = xgb.DMatrix(X, label=Y)
dtest = xgb.DMatrix(data_submit)

In [52]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['max_depth'] = 7
params['silent'] = 1
params['colsample_bytree'] = 0.8
params['eta'] = 0.005

In [53]:
cvp = xgb.cv(params, d_train, num_boost_round=100000, early_stopping_rounds=10, metrics=['logloss'], verbose_eval=1, stratified=True)
print("Best is {0} with logloss {1}".format(np.argmin(cvp['test-logloss-mean'] + cvp['test-logloss-std']), np.min(cvp['test-logloss-mean'] + cvp['test-logloss-std']))) 

[0]	train-logloss:0.692241+2.459e-05	test-logloss:0.69235+1.57692e-05
[1]	train-logloss:0.691407+0.000110376	test-logloss:0.691612+3.52641e-05
[2]	train-logloss:0.690519+0.000130103	test-logloss:0.690836+3.88616e-05
[3]	train-logloss:0.689698+0.000107397	test-logloss:0.69012+7.20062e-05
[4]	train-logloss:0.688865+0.00014585	test-logloss:0.689398+8.08826e-05
[5]	train-logloss:0.687996+0.000165395	test-logloss:0.688638+8.90206e-05
[6]	train-logloss:0.687182+0.000173506	test-logloss:0.687927+9.77218e-05
[7]	train-logloss:0.686404+0.000264681	test-logloss:0.687238+9.8625e-05
[8]	train-logloss:0.685604+0.000281788	test-logloss:0.686539+0.00011662
[9]	train-logloss:0.68479+0.000313767	test-logloss:0.68583+0.000129826
[10]	train-logloss:0.684043+0.000427876	test-logloss:0.68518+0.000151759
[11]	train-logloss:0.683221+0.000447609	test-logloss:0.684477+0.000152299
[12]	train-logloss:0.682462+0.000409127	test-logloss:0.683817+0.000219176
[13]	train-logloss:0.68172+0.000511559	test-logloss:0.6831

In [51]:
print("Best is {0} with logloss {1}".format(np.argmin(cvp['test-logloss-mean'] + cvp['test-logloss-std']), np.min(cvp['test-logloss-mean'] + cvp['test-logloss-std']))) 

Best is 979 with logloss 0.6074150539458709


In [ ]:
#clf = xgb.train(params, d_train, num_boost_round=466)

In [ ]:
#preds = clf.predict(dtest)
#submission = pd.DataFrame()
#submission["shot_id"] = data_submit.index
#submission["shot_made_flag"]= preds

#submission.to_csv("sub_xgb.csv",index=False)